Credits to Nick McCullum and FreeCodeCamp. Some information has been altered to fit my own experience because the IEX Cloud API has deprecated their sandbox API.
[Video Tutorial](https://www.youtube.com/watch?v=xfzGZB4HhEE&t=6104s)
# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [30]:
import numpy as np # fast way to process arrays and computational data
import pandas as pd # easy to work with tabular data
import requests
import xlsxwriter
import math
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import re

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [50]:
# # Run this when sp_500_stocks.csv is outdated.
# def substring_href(href):
#     """Get the ticker from the string of html element (a tag w/ href)"""
#     href_str = str(href)
#     start = href_str.find('l/')
#     end = href_str.find('">')
#     start += len('l/')
#     ticker = href_str[start:end]
#     return ticker

# url = 'https://www.slickcharts.com/sp500'

# browser = webdriver.Firefox()
# browser.get(url)
# time.sleep(3)
# html_source = browser.page_source
# browser.close()
# soup = BeautifulSoup(html_source, 'html.parser')
# soup = soup.find_all(href=re.compile("/symbol/"))

# stocks = []
# for i in range(0, len(soup), 2):
#     stock = substring_href(soup[i])
#     stocks.append(stock)
    
# d = {'Ticker': stocks}
# stocks_dataframe = pd.DataFrame(data=d)
# stocks_dataframe.to_csv('sp_500_stocks.csv', encoding='utf-8', index=False)

In [51]:
stocks = pd.read_csv('sp_500_stocks.csv') # csv -> pandas DataFrame
print(type(stocks))
stocks

<class 'pandas.core.frame.DataFrame'>


,Ticker
0,AAPL
1,MSFT
2,AMZN
3,NVDA
4,TSLA
...,...
502,NWS
503,FTRE
504,PHIN
505,SPY


## Acquiring an API Token

Now it's time to import our Finnhub API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `my_secrets.py` (because `secrets.py` interferes with the setup of NumPy) file that **doesn't** get pushed to your local Git repository.

In [3]:
from my_secrets import FINNHUB_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to Finnhub. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock

In [53]:
symbol = 'AAPL'
api_url = f'https://finnhub.io/api/v1/stock/profile2?symbol={symbol}&token={FINNHUB_API_TOKEN}' # retrieved from https://www.alphavantage.co/documentation/
company_data = requests.get(api_url).json()
company_data

{}

In [5]:
api_url2 = f'https://finnhub.io/api/v1/quote?symbol={symbol}&token={FINNHUB_API_TOKEN}'
price_data = requests.get(api_url2).json()
price_data

{'c': 190.68,
 'd': -1.13,
 'dp': -0.5891,
 'h': 192.67,
 'l': 190.245,
 'o': 191.41,
 'pc': 191.81,
 't': 1688760004}

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
market_cap = company_data['marketCapitalization']
price = price_data['c'] # current price

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns =['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

In [8]:
aapl_df = pd.DataFrame(
    [
        [symbol, price, market_cap, 'N/A']
    ],
    columns=my_columns
)
final_dataframe = pd.concat([final_dataframe, aapl_df], ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,190.68,2.999149e+06,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [55]:
final_dataframe = pd.DataFrame(columns=my_columns)

free_api_limit = 60
api_wait = 60
num_api_calls = 0
first_call = True

for stock in stocks['Ticker']:
    if num_api_calls % free_api_limit == 0 and first_call == False: # when all the free api call limits are used up, wait
        time.sleep(api_wait + 0.5)
        
    num_api_calls += 1    
    first_call = False
    print(f'{num_api_calls=} \n{stock=}')

    api_url = f'https://finnhub.io/api/v1/stock/profile2?symbol={stock}&token={FINNHUB_API_TOKEN}'
    company_data = requests.get(api_url).json()

    api_url2 = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_TOKEN}'
    price_data = requests.get(api_url2).json()

    try:
        market_cap = company_data['marketCapitalization']
        price = price_data['c']
    except KeyError:
        print(f"Finnhub does not have data for {stock}.")
        continue
    except:
        print("You may have an outdated S&P500 csv file. \nPlease scroll back above and run the scraper script to update.")
        print("If you have done that, and the error persists, you might have surpassed the api limit call.")
        break
    

    final_dataframe = pd.concat(
        [
            final_dataframe, 
            pd.DataFrame(
                [
                    [stock, price, market_cap, 'N/A']
                ], 
                columns=my_columns
            )
        ], ignore_index = True
    )
    

num_api_calls=1 
stock='AAPL'
num_api_calls=2 
stock='MSFT'
num_api_calls=3 
stock='AMZN'
num_api_calls=4 
stock='NVDA'
num_api_calls=5 
stock='TSLA'
num_api_calls=6 
stock='GOOGL'
num_api_calls=7 
stock='META'
num_api_calls=8 
stock='GOOG'
num_api_calls=9 
stock='BRK.B'
num_api_calls=10 
stock='UNH'
num_api_calls=11 
stock='JNJ'
num_api_calls=12 
stock='JPM'
num_api_calls=13 
stock='XOM'
num_api_calls=14 
stock='V'
num_api_calls=15 
stock='LLY'
num_api_calls=16 
stock='PG'
num_api_calls=17 
stock='AVGO'
num_api_calls=18 
stock='MA'
num_api_calls=19 
stock='HD'
num_api_calls=20 
stock='MRK'
num_api_calls=21 
stock='CVX'
num_api_calls=22 
stock='PEP'
num_api_calls=23 
stock='ABBV'
num_api_calls=24 
stock='COST'
num_api_calls=25 
stock='KO'
num_api_calls=26 
stock='ADBE'
num_api_calls=27 
stock='WMT'
num_api_calls=28 
stock='MCD'
num_api_calls=29 
stock='CSCO'
num_api_calls=30 
stock='CRM'
num_api_calls=31 
stock='PFE'
num_api_calls=32 
stock='TMO'
num_api_calls=33 
stock='BAC'
num_api_c

num_api_calls=268 
stock='PWR'
num_api_calls=269 
stock='EFX'
num_api_calls=270 
stock='AWK'
num_api_calls=271 
stock='MLM'
num_api_calls=272 
stock='ACGL'
num_api_calls=273 
stock='KDP'
num_api_calls=274 
stock='OKE'
num_api_calls=275 
stock='AVB'
num_api_calls=276 
stock='HPQ'
num_api_calls=277 
stock='ALB'
num_api_calls=278 
stock='EIX'
num_api_calls=279 
stock='GLW'
num_api_calls=280 
stock='XYL'
num_api_calls=281 
stock='CBRE'
num_api_calls=282 
stock='SBAC'
num_api_calls=283 
stock='IR'
num_api_calls=284 
stock='FTV'
num_api_calls=285 
stock='ES'
num_api_calls=286 
stock='CDW'
num_api_calls=287 
stock='WTW'
num_api_calls=288 
stock='TROW'
num_api_calls=289 
stock='CHD'
num_api_calls=290 
stock='STT'
num_api_calls=291 
stock='CAH'
num_api_calls=292 
stock='TSCO'
num_api_calls=293 
stock='WY'
num_api_calls=294 
stock='ULTA'
num_api_calls=295 
stock='ALGN'
num_api_calls=296 
stock='EBAY'
num_api_calls=297 
stock='FANG'
num_api_calls=298 
stock='GPC'
num_api_calls=299 
stock='MPWR'
n

In [56]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,190.68,2.999149e+06,N/A
1,MSFT,337.22,2.507395e+06,N/A
2,AMZN,129.78,1.331589e+06,N/A
3,NVDA,425.03,1.049824e+06,N/A
4,TSLA,274.43,8.698071e+05,N/A
...,...,...,...,...
499,ZION,28.98,4.291959e+03,N/A
500,LNC,26.08,4.422089e+03,N/A
501,NWL,8.87,3.673067e+03,N/A
502,NWS,19.91,1.134191e+04,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

However, for Finnhub Stock API has no batch API calls.

In [ ]:
# N/A

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [57]:
# keep prompting until we get numbers
while True:
    try:
        portfolio_size = float(input('Enter the value of your portfolio: '))
    except ValueError:
        print("That's not a number! \nPlease try again.")
        continue
    else:
        print("Portfolio size: ${:0.2f}".format(portfolio_size))
        break

Enter the value of your portfolio: 100000
Portfolio size: $100000.00


In [58]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

198.4126984126984

In [59]:
for i in range(0, len(final_dataframe.index)):
    current_price = final_dataframe.loc[i, 'Stock Price']
    formula = math.floor(position_size / current_price)
    final_dataframe.loc[i, 'Number of Shares to Buy'] = formula

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,190.68,2.999149e+06,1
1,MSFT,337.22,2.507395e+06,0
2,AMZN,129.78,1.331589e+06,1
3,NVDA,425.03,1.049824e+06,0
4,TSLA,274.43,8.698071e+05,0
...,...,...,...,...
499,ZION,28.98,4.291959e+03,6
500,LNC,26.08,4.422089e+03,7
501,NWL,8.87,3.673067e+03,22
502,NWS,19.91,1.134191e+04,9


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [60]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False) # 'Recommended Trades' is the sheet page name

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [61]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

# writer.close()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in a loop:

In [63]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [64]:
writer.close()